In [ ]:
!pip install -U spacy==2.1.0
!python -m spacy download en
!pip uninstall -y neuralcoref 
!pip install neuralcoref --no-binary neuralcoref
!pip install wikipedia

     |████████████████████████████████| 27.7 MB 1.3 MB/s 
     |████████████████████████████████| 2.1 MB 35.6 MB/s 
     |████████████████████████████████| 3.2 MB 8.5 MB/s 
     |████████████████████████████████| 82 kB 294 kB/s 
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.6
    Uninstalling preshed-3.0.6:
      Successfully uninstalled preshed-3.0.6
  Attempting uninstall: plac
    Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.3.3
    Uninstalling jsonschema-4.3.3:
      Successfully uninstalled jsonschema-4.3.3
  Attemptin

In [ ]:
import wikipedia
wikipedia.set_lang("en")

In [ ]:

articles = ['Nikola Tesla','Ahmed Zewail' , 'Ali Moustafa Mosharafa','Isaac Newton', 'Albert Einstein','Thomas Ediison']

text = [wikipedia.page(article).content for article in articles]

text



['Nikola Tesla ( TESS-lə; Serbian Cyrillic: Никола Тесла, pronounced [nǐkola têsla]; 10 July [O.S. 28 June] 1856 – 7 January 1943) was a Serbian-American inventor, electrical engineer, mechanical engineer, and futurist best known for his contributions to the design of the modern alternating current (AC) electricity supply system.Born and raised in the Austrian Empire, Tesla studied engineering and physics in the 1870s without receiving a degree, gaining practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he emigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His alternating current (AC) induction motor and related polypha

In [ ]:
import spacy
import neuralcoref

# Load SpaCy
nlp = spacy.load('en_core_web_sm')
# Add neural coref to SpaCy's pipe
neuralcoref.add_to_pipe(nlp)

def coref_resolution(text):
  doc = nlp(text)
  # fetches tokens with whitespaces from spacy document
  tok_list = list(token.text_with_ws for token in doc)
  for cluster in doc._.coref_clusters:
      # get tokens from representative cluster name
      cluster_main_words = set(cluster.main.text.split(' '))
      for coref in cluster:
            if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words 
                    # are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

  return "".join(tok_list)

100%|██████████| 40155833/40155833 [00:02<00:00, 19380763.25B/s]


In [ ]:
question = "when was Ali Moustafa Mosharafa born?"

In [ ]:
import re

def preprocessing(text):
    letters = []
    processed_paragraphs = []
    paragraphs = [ i.split('\n\n') for i in text ]

    for item in paragraphs:
      for paragraph in item:
        processed_paragraph = coref_resolution(paragraph)
        processed_paragraphs.append(processed_paragraph)
          
    for string in  processed_paragraphs:
      s = re.sub('[!#&()/;=@[\]^_`{|}~]', "", string)
      i = s.split()
      if len(i) > 5:
        letters.append(s)
    return letters    


In [ ]:
letters = preprocessing(text)

In [ ]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

In [ ]:


tokenized_corpus = [doc.split(" ") for doc in letters]
bm25 = BM25Okapi(tokenized_corpus)
tokenized_query = question.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
top_rank_para =  bm25.get_top_n(tokenized_query, letters, n=3)[0]
# candidate_document =letters[docu_4_cls2['doc_content']==top_rank_para ]
# candidate_document = list(candidate_document)[0]

top_rank_para

"Dr. Ali Moustafa Mosharafa' Egyptian Arabic: على مصطفى مشرفة 11 July 1898 – 16 January 1950 was an Egyptian theoretical physicist. Dr. Ali Moustafa Mosharafa' Egyptian Arabic: على مصطفى مشرفة 11 July 1898 – 16 January 1950 was professor of applied mathematics in the Faculty of Science at Cairo University, and also served as its first dean. Dr. Ali Moustafa Mosharafa' Egyptian Arabic: على مصطفى مشرفة 11 July 1898 – 16 January 1950 contributed to the development of the quantum theory as well as the theory of relativity."

In [ ]:
!pip install transformers
from transformers import BertForQuestionAnswering
import torch
from transformers import BertTokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, top_rank_para)

In [ ]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [ ]:
# Run our example through the model.
outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                             token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                             return_dict=True) 

start_scores = outputs.start_logits
end_scores = outputs.end_logits


In [ ]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')
    

In [ ]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])
# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]



In [ ]:
question = "when was Ali Moustafa Mosharafa born?"
print('Question: ' + question + '"')
print('Answer: "' + answer + '"')

Question: "when was Ali Moustafa Mosharafa born?"
Answer: "11 july 1898"
